In [4]:
import numpy as np
import bokeh.plotting as bkp
import bokeh.models as bkm
from bokeh.io import push_notebook
from ipywidgets import interact
bkp.output_notebook()

Loading BokehJS ...

# Solidification week 3 homework

### Mikko Karkkainen 11740378



The zone melting process is used to purify materials by taking advantage of the segregation of impurities to one end of the sample in directional solidification, as seen in Fig. 5.6. In zone melting, a segment of the sample of thickness $ \delta $ is melted, for example using an induction heater. The coil then traverses the sample beginning at one end. The heater produces sufficient stirring in the melted section that its composition can be considered to be uniform. The composition profile at some time during the first pass is shown schematically in the figure below. The initial composition of the sample is $ C_0 $, and the composition in the liquid is designated $ C_l^* $.

<img src="https://dev-plebbit.s3.amazonaws.com/uploads/topic/picture/17/zone_melting.png"/>
---

#### a) Write a solute balance for the change in composition in the liquid when the interface moves by an increment of $ dx $ as shown. Be sure to account for both the solute rejected at the solid-liquid-interface, and the dilution associated with previously unmelted solid.

Assuming no convection in the liquid, flux balance at the interface gives

$$ (C_0-C_S^*)dx=\delta \: dC_l^* $$

#### b) Rearrange this equation so that it can be integrated from the initial condition, where the composition in the liquid is $ C_0 $ and the solid-liquid interface is located at $ x^* = 0 $.


Let's rearrange:

$$  dx = \frac{\delta}{C_0 -C_S^*} \: dC^*_l $$ 

Substitute $ k_0C_l^* $ for  $ C_S^* $:

$$  dx = \frac{\delta}{C_0 - k_0C_l^*} \: dC^*_l $$

Define integral:

$$ \int^{x^*}_0 dx = \int_{C_0}^{C_l^*}\frac{\delta}{C_0 - k_0C_l^*} \: dC^*_l $$

---

#### c) Perform the integration to obtain the result

$$ \frac{C_l^*}{C_0} = \frac{1}{k_0} \Bigg[ 1 - (1-k_0)exp \bigg(-\frac{k_0x^*}{\delta} \bigg) \Bigg]$$

Performing the integral:

$$ x^* =  \frac{\delta}{k_0} \Big[ ln(C_0-k_0C_l^*) - ln(C_0-k_0C_0) \Big] $$

Rearranging:

$$ \frac{k_0x^*}{\delta} = ln \Bigg( \frac{C_0-k_0C_l^*}{C_0-k_0C_0} \Bigg) $$

exp from both sides:

$$ exp \Big( \frac{k_0x^*}{\delta} \Big) = \frac{C_0-k_0C_l^*}{C_0-k_0C_0}  $$

Rearrange:

$$ exp \Big( \frac{k_0x^*}{\delta} \Big) = \frac{1-\frac{k_0C_l^*}{C_0}}{1-k_0}  $$

Rearrange:

$$ (1-k_0)exp \Big( \frac{k_0x^*}{\delta} \Big) = 1-\frac{k_0C_l^*}{C_0}  $$

Rearrange:

$$ \frac{k_0C_l^*}{C_0}  = 1- (1-k_0)exp \Big( \frac{k_0x^*}{\delta} \Big) $$

Rearrange:

$$ \frac{C_l^*}{C_0}  = \frac{1}{k_0} \Big[ 1- (1-k_0)exp \Big( \frac{k_0x^*}{\delta} \Big) \Big] $$

---

#### d) Real zone melting processes perform multiple passes. One cannot find an analytical solution for the later passes. Sketch the solute profile that you would expect to see after the second and tenth passes. What would happen if this were a multicomponent aloy, whose constituents had different segregation coefficients.

Let's make a visualisation of the zone melting process. Let
s use the data for Al- 4.5wt%Cu from table B2, p. 387

In [ ]:
C_0=0.045
k=0.14
delta=10
l=100
